In [3]:
from dotenv import load_dotenv
from tqdm.notebook import tqdm
import openai
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [4]:
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [5]:
prompts_df = pd.read_csv(os.path.join('data', 'prompts_train.csv'))
prompts_df['prompt_text'] = prompts_df['prompt_text'].apply(lambda x: x.replace('\r\n', ' '))
prompts_df.head()

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 As the sequel to what has already ...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background The Third Wave experiment took pla...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


In [6]:
questions = prompts_df['prompt_question'].values
texts = prompts_df['prompt_text'].values


In [7]:
prompt_template = '''
You are a school literature teacher preparing a test in which you want to have your
students answer a question about a piece of writing. You want to have a baseline
to compare their answers with what you expect, in terms of ther expressing the 
main idea of the text, supportive details, in a coherent short essay of under 100
words. Your students may vary in grades 3 to 12, so you will need to consider the 
differing levels of reading, comprehension, and lexicon, and write an answer at each
level. (You will write three answers for each question.)

Here is the orignal text: ```{text}```

Here is the question they are to answer: ```{question}```

Respond with three answers, each appropriate for each of the three grade levels, grades 3-5, 
6-8, and 9-12. Use the following JSON template for your answers:

'''

json_template = '''
{"grades 3-5": <answer at elementary level>,
 "grades 6-8": <answer at junior high school level>,
 "grades 6-9": <answer at highs chool level>}
'''


In [8]:
GPT4_answers = []
for index in tqdm(range(len(questions))):
    prompt = prompt_template.format(text=texts[index], question=questions[index])
    prompt += json_template
    message = [{'role': 'user', 'content': prompt}]

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages = message,
        temperature=0
    )

    GPT4_answers.append(response.choices[0].message.content)
    
    

  0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
GPT4_answers

['{"grades 3-5": "Aristotle says a good tragedy story should have a few things. First, it should be a complex story, not a simple one. Second, it should make us feel pity and fear. Third, the main character should not be very good or very bad, but somewhere in the middle. They should have a big problem or mistake that causes them to go from happy to sad.",\n\n "grades 6-8": "According to Aristotle, an ideal tragedy should have three main elements. Firstly, it should have a complex plot, not a simple one. Secondly, it should imitate actions that cause feelings of pity and fear. Lastly, the main character should not be extremely good or bad, but somewhere in between. Their downfall should be caused by a mistake or weakness, not because they are evil.",\n\n "grades 9-12": "Aristotle outlines three key elements of an ideal tragedy. Firstly, the plot should be complex rather than simple. Secondly, the actions within the plot should evoke feelings of pity and fear, which are the distinctive 

In [10]:
GPT4_answers = list(map(json.loads, GPT4_answers))
GPT4_answers_df = pd.DataFrame(GPT4_answers)
GPT4_answers_df

,grades 3-5,grades 6-8,grades 9-12
0,Aristotle says a good tragedy story should hav...,"According to Aristotle, an ideal tragedy shoul...",Aristotle outlines three key elements of an id...
1,The ancient Egyptian government was like a pyr...,The ancient Egyptian government was structured...,The ancient Egyptian government was a theocrat...
2,"Mr. Jones, a teacher, started an experiment ca...",The Third Wave experiment was started by a tea...,"The Third Wave experiment, initiated by histor..."
3,The factory used spoiled meat in many ways. Th...,The factory had several methods to use or hide...,The factory employed a variety of methods to u...


In [11]:
prompts_df = pd.concat([prompts_df, GPT4_answers_df], axis=1)
prompts_df

,prompt_id,prompt_question,prompt_title,prompt_text,grades 3-5,grades 6-8,grades 9-12
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 As the sequel to what has already ...,Aristotle says a good tragedy story should hav...,"According to Aristotle, an ideal tragedy shoul...",Aristotle outlines three key elements of an id...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,The ancient Egyptian government was like a pyr...,The ancient Egyptian government was structured...,The ancient Egyptian government was a theocrat...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background The Third Wave experiment took pla...,"Mr. Jones, a teacher, started an experiment ca...",The Third Wave experiment was started by a tea...,"The Third Wave experiment, initiated by histor..."
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",The factory used spoiled meat in many ways. Th...,The factory had several methods to use or hide...,The factory employed a variety of methods to u...


In [12]:
prompts_df.to_csv(os.path.join('data', 'prompts_GPT4_answers.csv'), index=False)